In [1]:
#!pip install netCDF4
#from netCDF4 import Dataset

#dataset = Dataset('sipexII.nc')
#print(dataset)  # Prints summary info
#print(dataset.variables.keys())  # Prints variable names

In [48]:
import numpy as np
import pandas as pd 
from netCDF4 import Dataset
import os
import matplotlib.pyplot as plt
from datetime import datetime

def process_wave_data(file_path="sipexII.nc"):
    try:
        # Open the netCDF file
        print(f"Opening file: {file_path}")
        nc = Dataset(file_path, 'r')
        
        # Print variables and dimensions for debugging
        print("Variables:", list(nc.variables.keys()))
        print("Dimensions:", list(nc.dimensions.keys()))
        
        # Extract time string using a simpler approach
        if 'time_str_utc' in nc.variables:
            time_chars = nc.variables['time_str_utc'][:]
            print(f"time_str_utc shape: {time_chars.shape}")
            print(f"time_str_utc dtype: {time_chars.dtype}")
            
            # Print a sample raw data for debugging
            print("Sample raw time_str_utc data (first row):")
            print(time_chars[0, :20])  # First 20 chars of first row
            
            # Convert character array to strings - approach for bytes
            time_strings = []
            for i in range(time_chars.shape[0]):
                # For bytes objects
                if time_chars.dtype.kind == 'S':  # String/bytes type
                    # Join all bytes together and decode
                    try:
                        # Try to decode the whole array at once
                        full_str = b''.join(filter(None, time_chars[i])).decode('utf-8', errors='replace').strip()
                        time_strings.append(full_str)
                    except:
                        # Fallback: try to decode each byte individually
                        chars = []
                        for c in time_chars[i]:
                            if c:  # Skip empty bytes
                                try:
                                    decoded = c.decode('utf-8', errors='replace')
                                    chars.append(decoded)
                                except:
                                    pass
                        time_strings.append(''.join(chars).strip())
                else:
                    # For numeric arrays (like uint8)
                    chars = []
                    for c in time_chars[i]:
                        if c > 0:  # Skip nulls
                            chars.append(chr(c))
                    time_strings.append(''.join(chars).strip())
            
            time_str = np.array(time_strings)
            print(f"Converted time_str shape: {time_str.shape}")
            print(f"Sample times: {time_str[:5]}")
        else:
            time_str = None
            print("Warning: 'time_str_utc' variable not found in the file")
        
        # Extract other variables
        psd = nc.variables['psd'][:, 2:7, :]  # Select sensors 2-6
        bin_vals = nc.variables['bin'][:]
        time = nc.variables['time'][:]
        moments = nc.variables['moments'][:, 2:7, :]
        sensor = nc.variables['sensor'][2:7]
        lat = nc.variables['lat'][:, 2:7]
        lon = nc.variables['lon'][:, 2:7]
        
        # Print shapes for debugging
        print(f"Shapes - psd: {psd.shape}, moments: {moments.shape}, lat: {lat.shape}, lon: {lon.shape}")
        
        # Calculate derived values
        T = 1/bin_vals
        
        # Create numpy arrays to work with (not masked arrays)
        psd_arr = np.array(psd)
        moments_arr = np.array(moments)
        lat_arr = np.array(lat)
        lon_arr = np.array(lon)
        
        # Use NaN as fill value for missing data
        fill_value = np.nan
        
        # Function to set values to NaN
        def set_nan(arr, indices):
            arr_copy = arr.copy()
            if isinstance(indices, tuple):
                arr_copy[indices] = fill_value
            else:
                for idx in indices:
                    arr_copy[idx] = fill_value
            return arr_copy

        # Remove data whilst transmitting from ship
        lon_arr = set_nan(lon_arr, (slice(0, 4), 0))
        lat_arr = set_nan(lat_arr, (slice(0, 4), 0))
        lon_arr = set_nan(lon_arr, (slice(0, 4), 1))
        lat_arr = set_nan(lat_arr, (slice(0, 4), 1))
        lon_arr = set_nan(lon_arr, (slice(0, 6), 2))
        lat_arr = set_nan(lat_arr, (slice(0, 6), 2))
        lon_arr = set_nan(lon_arr, (slice(0, 4), 3))
        lat_arr = set_nan(lat_arr, (slice(0, 4), 3))
        lon_arr = set_nan(lon_arr, (slice(0, 12), 4))
        lat_arr = set_nan(lat_arr, (slice(0, 12), 4))
        
        psd_arr = set_nan(psd_arr, (slice(0, 4), 0, slice(None)))
        psd_arr = set_nan(psd_arr, (slice(0, 4), 1, slice(None)))
        psd_arr = set_nan(psd_arr, (slice(0, 6), 2, slice(None)))
        psd_arr = set_nan(psd_arr, (slice(0, 4), 3, slice(None)))
        psd_arr = set_nan(psd_arr, (slice(0, 12), 4, slice(None)))
        
        moments_arr = set_nan(moments_arr, (slice(0, 4), 0, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(0, 4), 1, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(0, 6), 2, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(0, 4), 3, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(0, 12), 4, slice(None)))
        
        # Fill bad values with missing values
        # Spikes in sensor 0
        psd_arr = set_nan(psd_arr, (85, 0, slice(None)))
        psd_arr = set_nan(psd_arr, (100, 0, slice(None)))
        moments_arr = set_nan(moments_arr, (85, 0, slice(None)))
        moments_arr = set_nan(moments_arr, (100, 0, slice(None)))
        lat_arr = set_nan(lat_arr, (85, 0))
        lat_arr = set_nan(lat_arr, (100, 0))
        
        # Spikes in sensor 1
        psd_arr = set_nan(psd_arr, (42, 1, slice(None)))
        psd_arr = set_nan(psd_arr, (slice(73, 75), 1, slice(None)))
        moments_arr = set_nan(moments_arr, (42, 1, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(73, 75), 1, slice(None)))
        lat_arr = set_nan(lat_arr, (42, 1))
        lat_arr = set_nan(lat_arr, (slice(73, 75), 1))
        
        # Gyro issues in sensor 3
        psd_arr = set_nan(psd_arr, (slice(8, 11), 3, slice(None)))
        moments_arr = set_nan(moments_arr, (slice(8, 11), 3, slice(None)))
        lat_arr = set_nan(lat_arr, (slice(8, 11), 3))
        
        # Spikes in sensor 4
        psd_arr = set_nan(psd_arr, (318, 4, slice(None)))
        moments_arr = set_nan(moments_arr, (318, 4, slice(None)))
        lat_arr = set_nan(lat_arr, (318, 4))
        
        # Calculate Significant Wave Height (Hs)
        num_times = psd_arr.shape[0]
        Hs = np.full((num_times, 5), fill_value)
        for i in range(5):
            for j in range(num_times):
                if not np.isnan(moments_arr[j, i, 2]):
                    Hs[j, i] = 4 * np.sqrt(moments_arr[j, i, 2])
                    
        # Filter out unrealistic Hs values (> 20m)
        unrealistic_mask = Hs > 20.0
        if np.any(unrealistic_mask):
            unrealistic_count = np.sum(unrealistic_mask)
            print(f"Found {unrealistic_count} data points with Hs > 20m. Setting these to NaN.")
            
            # Set Hs to NaN where values are unrealistic
            Hs[unrealistic_mask] = fill_value
            
            # Also set corresponding Tp values to NaN (will calculate later)
            # and lat/lon values
            for i in range(5):
                for j in range(num_times):
                    if unrealistic_mask[j, i]:
                        # Set lat and lon to NaN for the same indices
                        lat_arr[j, i] = fill_value
                        lon_arr[j, i] = fill_value
                        # We'll set Tp to NaN when we calculate it
        
        # No waves condition
        Hs[41:43, 4] = 0
        
        # Calculate Peak Period (Tp)
        Tp = np.full((num_times, 5), fill_value)
        for i in range(5):
            for j in range(num_times):
                # Skip if Hs is NaN (unrealistic values) or all psd values are NaN
                if np.isnan(Hs[j, i]) or np.all(np.isnan(psd_arr[j, i, :])):
                    continue
                    
                # Find index of max value, ignoring NaNs
                valid_indices = ~np.isnan(psd_arr[j, i, :])
                if np.any(valid_indices):
                    max_idx = np.nanargmax(psd_arr[j, i, :])
                    Tp[j, i] = T[max_idx]
                
        # Process each sensor and create CSV files
        for sensor_idx in range(5):
                    
            # Original sensor number (adding 2 because we selected 2:7)
            original_sensor_id = sensor[sensor_idx]
            
            # Create data for DataFrame
            data = {
                'DD/MM/YYYY HH:MM (UTC)': time_str[:],
                'Latitude (decimal degrees)': lat_arr[:, sensor_idx],
                'Longitude (decimal degrees)': lon_arr[:, sensor_idx],
                'Significant Wave Height (m)': Hs[:, sensor_idx],
                'Peak Period (s)': Tp[:, sensor_idx]
            }
            
            # Create DataFrame
            df = pd.DataFrame(data)
            
            # Filter out rows with missing lat/lon
            df_filtered = df.dropna(subset=['Latitude (decimal degrees)', 'Longitude (decimal degrees)'])
            
            # Save to CSV
            csv_filename = f'SIPEX{int(original_sensor_id)}_data.csv'
            df_filtered.to_csv(csv_filename, index=False)
            print(f"Saved {csv_filename} with {len(df_filtered)} rows")
            
            # Create a plot for verification
            try:
                # Filter data to include only non-null values for both wave height and period
                plot_df = df_filtered.dropna(subset=['Significant Wave Height (m)', 'Peak Period (s)', 'DD/MM/YYYY HH:MM (UTC)'])
                
                if len(plot_df) == 0:
                    print(f"No valid data to plot for sensor {int(original_sensor_id)}")
                    return
                
                # Print sample of date strings to debug
                print("Sample date strings:", plot_df['DD/MM/YYYY HH:MM (UTC)'].iloc[:5].tolist())
                
                # Cap Hs values to reasonable range (0-20m)
                plot_df['Significant Wave Height (m)'] = plot_df['Significant Wave Height (m)'].clip(upper=20.0)
                
                # Try simpler approach: just use the row index for x-axis
                # Create figure and primary axis for wave height
                fig, ax1 = plt.subplots(figsize=(14, 7))
                
                # Plot wave height on the primary axis
                color1 = 'tab:blue'
                ax1.set_ylabel('Significant Wave Height (m)', color=color1, fontsize=12)
                line1 = ax1.plot(range(len(plot_df)), plot_df['Significant Wave Height (m)'], 
                         color=color1, label='Significant Wave Height', marker='o', markersize=4)
                ax1.tick_params(axis='y', labelcolor=color1)
                
                # Set y-axis limit for wave height
                ax1.set_ylim(0, 20)
                
                # Create a second y-axis that shares the same x-axis
                ax2 = ax1.twinx()
                
                # Plot peak period on the secondary axis
                color2 = 'tab:red'
                ax2.set_ylabel('Peak Period (s)', color=color2, fontsize=12)
                line2 = ax2.plot(range(len(plot_df)), plot_df['Peak Period (s)'], 
                         color=color2, label='Peak Period', marker='x', markersize=4)
                ax2.tick_params(axis='y', labelcolor=color2)
                
                # Format x-axis with date strings at select positions
                if len(plot_df) > 20:
                    # Select a subset of indices for x-ticks
                    n_ticks = min(15, len(plot_df))
                    step = max(1, len(plot_df) // n_ticks)
                    tick_positions = list(range(0, len(plot_df), step))
                    
                    # Make sure the last position is included
                    if len(plot_df) - 1 not in tick_positions:
                        tick_positions.append(len(plot_df) - 1)
                        
                    # Set tick positions and labels
                    ax1.set_xticks(tick_positions)
                    ax1.set_xticklabels([plot_df['DD/MM/YYYY HH:MM (UTC)'].iloc[i] for i in tick_positions], rotation=45, ha='right')
                else:
                    # If few points, show all date labels
                    ax1.set_xticks(range(len(plot_df)))
                    ax1.set_xticklabels(plot_df['DD/MM/YYYY HH:MM (UTC)'], rotation=45, ha='right')
                
                # Add a title
                plt.title(f'Wave Data - Sensor {int(original_sensor_id)}', fontsize=14)
                
                # Add legend for both lines
                ax1.legend(line1 + line2, ['Significant Wave Height', 'Peak Period'], 
                           loc='upper right', fontsize=11)
                
                # Add grid for better readability
                ax1.grid(True, alpha=0.3)
                
                # Add x-axis label
                ax1.set_xlabel('Date/Time (UTC)', fontsize=12)
                
                # Adjust layout to make room for labels
                fig.tight_layout()
                
                # Save figure
                plt.savefig(f'SIPEX{int(original_sensor_id)}_plot.png', dpi=300)
                plt.close()
                
                print(f"Plot saved for sensor {int(original_sensor_id)} with {len(plot_df)} data points")
            except Exception as e:
                print(f"Error creating plot: {e}")
                import traceback
                traceback.print_exc()
        
        # Close the netCDF file
        nc.close()
        print("Data processing completed.")
        return True
        
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# Call the function to process the data
if __name__ == "__main__":
    success = process_wave_data()
    
    if success:
        print("Script completed successfully.")
    else:
        print("Script failed. Check error messages above.")

Opening file: sipexII.nc
Variables: ['time', 'sensor', 'bin', 'moments_header', 'orientation_header', 'direction_header', 'sd_gyro_header', 'time_header', 'psd', 'moments', 'direction', 'dir_ratio', 'orientation', 'sd_gyro', 'lat', 'lon', 'temp', 'volts', 'elev', 'sd_acc', 'sd_yaw', 'acc_removed', 'flat', 'max_flat', 'spike', 'max_spike', 'acc_flag', 'imu_flag', 'power_flag', 'file_id', 'time_str', 'time_str_utc']
Dimensions: ['time', 'sensor', 'bin', 'moments_header', 'orientation_header', 'direction_header', 'sd_gyro_header', 'time_header', 'chid']
time_str_utc shape: (325, 100)
time_str_utc dtype: |S1
Sample raw time_str_utc data (first row):
[b'2' b'2' b'/' b'0' b'9' b'/' b'2' b'0' b'1' b'2' b' ' b'1' b'4' b':'
 b'0' b'0' -- -- -- --]
Converted time_str shape: (325,)
Sample times: ['22/09/2012 14:00' '22/09/2012 17:00' '22/09/2012 20:00'
 '22/09/2012 23:00' '23/09/2012 2:00']
Shapes - psd: (325, 5, 55), moments: (325, 5, 6), lat: (325, 5), lon: (325, 5)
Found 997 data points with H